# Agent with memory 



In [1]:
from zork import go
from zork import env

# Setup environment
import textworld
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import OpenAI, LLMChain

from dotenv import load_dotenv
load_dotenv()

# Setup the language model
llm = OpenAI(
    model_name="text-davinci-003",
    temperature=0.0,
    max_tokens=100,
    # stop=["\n","\r"],
)


WARNING! stop is not default parameter.
                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.


In [2]:

# Tool for sending commmands to the game environment and getting back templated world state
def send_command(command):
    """Send a command to the game and receive feedback."""
    game_state, score, done = env.step(command)
    description = "" #if game_state.description == game_state.feedback else f"{game_state.description}"
    templated_feedback = f"""{description}{game_state.feedback}
(Score: {game_state.score}/{game_state.max_score}, Moves: {game_state.moves}, DONE: {done})
"""
    return templated_feedback

# tools = [Tool("Play", send_command, send_command.__doc__)]
tools = []

In [3]:

# Setup the agent with prompt and tools and memory
prefix = """You are playing a text adventure game. Explore the world and discover its secrets!
You have access to the following tools:"""
suffix = """
---
History:{chat_history}
---
Question: {input}
Respond with the next command.
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=[
        "input", 
        "chat_history", 
        "agent_scratchpad"]
)


In [4]:
print(prompt.template)

You are playing a text adventure game. Explore the world and discover its secrets!
You have access to the following tools:



Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of []
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


---
History:{chat_history}
---
Question: {input}
Respond with the next command.
{agent_scratchpad}


In [5]:
prompt.template = """You are playing a text adventure game. Explore the world and discover its secrets!

Use the following format:

Game: the input from the game
Thought: you should always think about what to do
... (this Thought can repeat N times)
Thought: I now know the next command
Final Answer: the next command to send to the game


---
History:{chat_history}
---
Game: {input}
{agent_scratchpad}"""

In [36]:


memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    human_prefix="Game: ",
    ai_prefix="Command: ",
    )

llm_chain = LLMChain(llm=llm, prompt=prompt,
 verbose=True
 )

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools, 
    memory=memory,
    verbose=True,
    max_iterations=5,
)

def npc(scene):
    """NPC agent that plays the game."""
    print(scene)
    command = agent_executor.run(scene)
    if command == "Agent stopped due to max iterations.":
        return "Look around"
    return agent_executor.run(scene)

In [35]:
# go(env, npc)

Okay, this isn't working very well. The context was a lot better when the agent was using a regular MRKL pattern and had the "Play" tool.

Let's try another way of getting around the context length. Let's give the agent a tool to play with, and then wrap that function in the `go` loop. That way it will advance the environment as it goes, but when it errors out it will start over in the same environment. 

In [39]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

tools = [Tool("Play", send_command, send_command.__doc__)]

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=[
        "input", 
        "chat_history", 
        "agent_scratchpad"
        ]
)


memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=40,
    memory_key="chat_history",
    human_prefix="Game: ",
    ai_prefix="Command: ",
    )

llm_chain = LLMChain(llm=llm, prompt=prompt,
#  verbose=True
 )

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools, 
    memory=memory,
    verbose=True,
    max_iterations=20,
)

def npc(scene):
    """NPC agent that plays the game."""
    print(scene)
    command = agent_executor.run(scene)
    if "agent" in command.lower():
        return "Look around"
    return agent_executor.run(scene)

In [40]:
go(env, npc)

############################################################ 1
West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.

Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


(Score: 0/350, Moves: 0, DONE: False)



> Entering new AgentExecutor chain...

Thought: What can I do here?
Action: Play
Action Input: open mailbox
Observation: Opening the small mailbox reveals a leaflet.


(Score: 0/350, Moves: 1, DONE: False)

Thought: What does the leaflet say?
Action: Play
Action Input: read leaflet
Observation: (Taken)
"WELCOME TO ZORK!

ZORK is a game of adventure, danger, and low cunning. In it you will explore some of the most amazing territory ever seen by mortals. No computer should be withou

TypeError: str.format() argument after * must be an iterable, not float

Well, that works better, kind of. It's possible for it to break out of previous loops, but also for it to get caught in more loops. And it's probably more expensive on tokens than anything so far.